<h1>PART 1<h1>

In [1]:
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner

In [2]:
!pip install beautifulsoup4
!pip install lxml
!pip install html5lib

In [3]:
import requests 
from bs4 import BeautifulSoup 

In [4]:
page = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
contents = page.content


In [16]:
soup = BeautifulSoup(contents, 'lxml') #used BeautifulSoup to retrieve the data form wikipedia
links =soup.find_all("a")


In [17]:
# by using soup i selected all the tables and chose the first table (the one that i need)

a = soup.find_all("table")[0] 
data = pd.read_html(str(a))
df = data[0]

print(df.shape)
print(df.head())


(288, 3)
  Postcode           Borough     Neighbourhood
0      M1A      Not assigned      Not assigned
1      M2A      Not assigned      Not assigned
2      M3A        North York         Parkwoods
3      M4A        North York  Victoria Village
4      M5A  Downtown Toronto      Harbourfront


In [18]:
# i got rid of the rows where Borough = Not assigned 

df=df[df.Borough != 'Not assigned']
df = df.sort_values(by=['Postcode','Borough'])

df.reset_index(inplace=True)
df.drop('index',axis=1,inplace=True)

print(df.shape)
print(df.head())

(211, 3)
  Postcode      Borough   Neighbourhood
0      M1B  Scarborough           Rouge
1      M1B  Scarborough         Malvern
2      M1C  Scarborough  Highland Creek
3      M1C  Scarborough      Rouge Hill
4      M1C  Scarborough      Port Union


In [19]:
# here i combined the Neighborhoods with the same Postal

dfNew = df.groupby(['Postcode', 'Borough'], sort=False).agg( ','.join)
dfNew = dfNew.groupby(['Postcode','Borough'])['Neighbourhood'].apply(list)
dfNew = dfNew.sample(frac=1).reset_index()
dfNew['Neighbourhood']= dfNew['Neighbourhood'].str.join(', ')


In [20]:
# here i replace the Neighbourhood = Not assigned with corresponding Borough value

dfNew['Neighbourhood'].loc[dfNew['Neighbourhood'] == 'Not assigned'] = dfNew['Borough']


In [21]:
dfNew.shape

(103, 3)

<h1> PART 2 <h1>

In [22]:
# first i downloaded the data
geodata = pd.read_csv('http://cocl.us/Geospatial_data')


In [23]:
# in order to merge 2 dataframes it is mandatory to have at least 1 column with the same name 

geodata.rename(columns = {'Postal Code':'Postcode'}, inplace = True)

In [24]:
#finally i have created my final dataframe which contain all the necessary information 

Data = pd.merge(dfNew, geodata)
Data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie...",43.688905,-79.554724
1,M2L,North York,"Silver Hills,York Mills",43.757490,-79.374714
2,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
3,M6E,York,Caledonia-Fairbanks,43.689026,-79.453512
4,M4V,Central Toronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",43.686412,-79.400049





<h1> PART 3 <h1>
    
    
    

In [25]:
#comments


In [26]:
# i start with downloading the necessary libraries

import json
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import requests 
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium 
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [27]:
Data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie...",43.688905,-79.554724
1,M2L,North York,"Silver Hills,York Mills",43.757490,-79.374714
2,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
3,M6E,York,Caledonia-Fairbanks,43.689026,-79.453512
4,M4V,Central Toronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",43.686412,-79.400049


In [28]:
#here I define a user agent and find the geolocation of Toronto

address = 'Toronto, ON'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [29]:
# here I present general map of Toronto with all its Borough and Neighbourhoods


map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for latitute, longitute, Borough, Neighborhood in zip(Data['Latitude'], Data['Longitude'], Data['Borough'], Data['Neighbourhood']):
    label = '{}, {}'.format(Neighborhood, Borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latitute, longitute],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

<h3> Important!  Please read the below cell. <h3>
    
<h4> As was also mentioned in the Instructions section of the Assignment,for illustration purposes, let's simplify the above map and segment and cluster only the neighborhoods in "DOWNTOWN TORONTO" only.  <h4>
    

In [30]:
# here i slice the original dataframe and create a new dataframe of the Downtown Toronto data 
# and get the geographical coordinates of Downtown Toronto

toronto_data = Data[Data['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
print(toronto_data.head())

address = 'Downtown Toronto, ON'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

  Postcode           Borough           Neighbourhood   Latitude  Longitude
0      M5E  Downtown Toronto             Berczy Park  43.644771 -79.373306
1      M5H  Downtown Toronto  Adelaide,King,Richmond  43.650571 -79.384568
2      M6G  Downtown Toronto                Christie  43.669542 -79.422564
3      M4Y  Downtown Toronto    Church and Wellesley  43.665860 -79.383160
4      M4W  Downtown Toronto                Rosedale  43.679563 -79.377529
The geograpical coordinate of Downtown Toronto are 43.655115, -79.380219.


In [31]:
# create map of Manhattan using latitude and longitude values
map_downtoronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for latitute, longitute, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latitute, longitute],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtoronto)  
    
map_downtoronto

In [32]:
# now i will define my Fourquare credentials for future reference

CLIENT_ID = '1CIOCU1T1I5Z4WJU42VHG4KPO0HAYWUN5H33LXU5IBMOI3OG' # your Foursquare ID
CLIENT_SECRET = 'O1II2ITGW4BCXCL2AKZ3NA0W1G54KPEM2NRVJ5HJP2UDHO01' # your Foursquare Secret
VERSION = '20180604'


In [33]:
toronto_data.loc[0, 'Neighbourhood']

'Berczy Park'

Lets explore the neighbourhoods in Downtown Toronto

In [47]:
LIMIT = 100

radius = 500

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


In [48]:
# here i create new dataframe which contain 
downtowntoronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )


Berczy Park
Adelaide,King,Richmond
Christie
Church and Wellesley
Rosedale
Central Bay Street
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Design Exchange,Toronto Dominion Centre
St. James Town
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
Harbourfront,Regent Park
Stn A PO Boxes 25 The Esplanade
Commerce Court,Victoria Hotel
First Canadian Place,Underground city
Ryerson,Garden District
Harbourfront East,Toronto Islands,Union Station
Cabbagetown,St. James Town


In [49]:
#Let's check the size of the resulting dataframe

print(downtowntoronto_venues.shape)
downtowntoronto_venues.head()

(1294, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Berczy Park,43.644771,-79.373306,LCBO,43.642944,-79.372440,Liquor Store
1,Berczy Park,43.644771,-79.373306,The Keg Steakhouse + Bar,43.646676,-79.374822,Steakhouse
2,Berczy Park,43.644771,-79.373306,Sony Centre for the Performing Arts,43.646292,-79.376022,Concert Hall
3,Berczy Park,43.644771,-79.373306,Fresh On Front,43.647815,-79.374453,Vegetarian / Vegan Restaurant
4,Berczy Park,43.644771,-79.373306,Hockey Hall Of Fame (Hockey Hall of Fame),43.646974,-79.377323,Museum


In [51]:
#Let's check how many venues were returned for each neighborhood

downtowntoronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Berczy Park,56,56,56,56,56,56
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara",15,15,15,15,15,15
"Cabbagetown,St. James Town",43,43,43,43,43,43
Central Bay Street,86,86,86,86,86,86
"Chinatown,Grange Park,Kensington Market",100,100,100,100,100,100
Christie,16,16,16,16,16,16
Church and Wellesley,87,87,87,87,87,87
"Commerce Court,Victoria Hotel",100,100,100,100,100,100


In [52]:
print('There are {} uniques categories.'.format(len(downtowntoronto_venues['Venue Category'].unique())))

There are 208 uniques categories.


In [60]:
#lets analize neighbourhoods 

downtoronto_onehot = pd.get_dummies(downtowntoronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
downtoronto_onehot['Neighbourhood'] = downtowntoronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [downtoronto_onehot.columns[-1]] + list(downtoronto_onehot.columns[:-1])
downtoronto_onehot = downtoronto_onehot[fixed_columns]

print(downtoronto_onehot.shape)
downtoronto_onehot.head()

(1294, 209)


,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Berczy Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Berczy Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Berczy Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Berczy Park,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,Berczy Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [61]:
#Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [62]:
downtoronto_grouped = downtoronto_onehot.groupby('Neighbourhood').mean().reset_index()
downtoronto_grouped

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide,King,Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.000000,...,0.00,0.0,0.00,0.020000,0.000000,0.00,0.010000,0.000000,0.01,0.000000
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.0,0.00,0.017857,0.000000,0.00,0.000000,0.000000,0.00,0.000000
2,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.000000,0.066667,0.066667,0.066667,0.133333,0.133333,0.133333,0.000000,0.000000,...,0.00,0.0,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000
3,"Cabbagetown,St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.0,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000
4,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011628,0.000000,...,0.00,0.0,0.00,0.011628,0.000000,0.00,0.011628,0.000000,0.00,0.011628
5,"Chinatown,Grange Park,Kensington Market",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.0,0.00,0.060000,0.000000,0.04,0.010000,0.000000,0.00,0.000000
6,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.0,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000
7,Church and Wellesley,0.011494,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011494,0.000000,...,0.00,0.0,0.00,0.000000,0.011494,0.00,0.000000,0.011494,0.00,0.022989
8,"Commerce Court,Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,...,0.00,0.0,0.00,0.020000,0.000000,0.00,0.010000,0.000000,0.00,0.000000
9,"Design Exchange,Toronto Dominion Centre",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,...,0.00,0.0,0.01,0.010000,0.000000,0.00,0.010000,0.000000,0.00,0.000000


In [63]:
downtoronto_grouped.shape

(18, 209)

In [65]:
#Let's print each neighborhood along with the top 5 most common venues
num_top_venues = 5

for hood in downtoronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = downtoronto_grouped[downtoronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
              venue  freq
0       Coffee Shop  0.07
1              Café  0.05
2               Bar  0.04
3   Thai Restaurant  0.04
4  Asian Restaurant  0.03


----Berczy Park----
          venue  freq
0   Coffee Shop  0.07
1  Cocktail Bar  0.05
2          Café  0.04
3    Steakhouse  0.04
4   Cheese Shop  0.04


----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara----
              venue  freq
0    Airport Lounge  0.13
1   Airport Service  0.13
2  Airport Terminal  0.13
3     Boat or Ferry  0.07
4           Airport  0.07


----Cabbagetown,St. James Town----
                venue  freq
0         Coffee Shop  0.07
1              Bakery  0.07
2                 Pub  0.05
3  Italian Restaurant  0.05
4         Pizza Place  0.05


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.14
1  Italian Restaurant  0.06
2                Café  0.05
3      Sandwich Place  0.05
4        Burger Jo

<h4> Let's put that into a pandas dataframe <h4>

In [66]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [74]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = downtoronto_grouped['Neighbourhood']

for ind in np.arange(downtoronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtoronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Berczy Park,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"CN Tower,Bathurst Quay,Island airport,Harbourf...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Cabbagetown,St. James Town",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Central Bay Street,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [126]:
# set number of clusters
kclusters = 5

downtoronto_grouped_clustering = downtoronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtoronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
print(kmeans.labels_ )
kmeans.labels_.shape

[1 1 0 1 1 3 4 1 1 1 1 3 1 1 2 1 1 1]


(18,)

In [156]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Label', kmeans.labels_)

downtoronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
downtoronto_merged = downtoronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

downtoronto_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Label,Cluster Labe,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,1,1,Coffee Shop,Cocktail Bar,Farmers Market,Seafood Restaurant,Bakery,Beer Bar,Cheese Shop,Steakhouse,Café,Restaurant
1,M5H,Downtown Toronto,"Adelaide,King,Richmond",43.650571,-79.384568,1,1,Coffee Shop,Café,Bar,Thai Restaurant,Steakhouse,American Restaurant,Burger Joint,Hotel,Asian Restaurant,Sushi Restaurant
2,M6G,Downtown Toronto,Christie,43.669542,-79.422564,4,4,Grocery Store,Café,Park,Athletics & Sports,Nightclub,Baby Store,Italian Restaurant,Diner,Restaurant,Convenience Store
3,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,1,1,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Gay Bar,Yoga Studio,Men's Store,Mediterranean Restaurant,Hotel,Gym
4,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,2,2,Park,Playground,Trail,Building,Yoga Studio,Dessert Shop,Ethiopian Restaurant,Electronics Store,Dumpling Restaurant,Donut Shop


<h4>
Lets visualize the resulting clusters <h4>

In [157]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map

markers_colors = []
for lat, lon, poi, cluster in zip(downtoronto_merged['Latitude'], downtoronto_merged['Longitude'], downtoronto_merged['Neighbourhood'], downtoronto_merged['Cluster Label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<h3> Cluster 1 <h3>

In [159]:
downtoronto_merged.loc[downtoronto_merged['Cluster Label'] == 0, downtoronto_merged.columns[[1] + list(range(5, downtoronto_merged.shape[1]))]]




,Borough,Cluster Label,Cluster Labe,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Downtown Toronto,0,0,Airport Lounge,Airport Service,Airport Terminal,Harbor / Marina,Coffee Shop,Sculpture Garden,Boat or Ferry,Bar,Boutique,Airport Gate


<h3> Cluster 2 <h3>

In [162]:
downtoronto_merged.loc[downtoronto_merged['Cluster Label'] == 1, downtoronto_merged.columns[[1] + list(range(5, downtoronto_merged.shape[1]))]]


,Borough,Cluster Label,Cluster Labe,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,1,1,Coffee Shop,Cocktail Bar,Farmers Market,Seafood Restaurant,Bakery,Beer Bar,Cheese Shop,Steakhouse,Café,Restaurant
1,Downtown Toronto,1,1,Coffee Shop,Café,Bar,Thai Restaurant,Steakhouse,American Restaurant,Burger Joint,Hotel,Asian Restaurant,Sushi Restaurant
3,Downtown Toronto,1,1,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Gay Bar,Yoga Studio,Men's Store,Mediterranean Restaurant,Hotel,Gym
5,Downtown Toronto,1,1,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Burger Joint,Ice Cream Shop,Middle Eastern Restaurant,Spa,Chinese Restaurant,Bar
7,Downtown Toronto,1,1,Coffee Shop,Café,Hotel,Restaurant,Gastropub,Steakhouse,Deli / Bodega,Italian Restaurant,Seafood Restaurant,Bar
8,Downtown Toronto,1,1,Coffee Shop,Café,Hotel,Restaurant,Clothing Store,Cosmetics Shop,Breakfast Spot,Bakery,Italian Restaurant,Gastropub
11,Downtown Toronto,1,1,Coffee Shop,Pub,Bakery,Café,Park,Breakfast Spot,Restaurant,Theater,Mexican Restaurant,Health Food Store
12,Downtown Toronto,1,1,Coffee Shop,Restaurant,Café,Italian Restaurant,Beer Bar,Seafood Restaurant,Cocktail Bar,Hotel,Cheese Shop,Park
13,Downtown Toronto,1,1,Coffee Shop,Café,Hotel,American Restaurant,Restaurant,Gastropub,Deli / Bodega,Steakhouse,Gym,Italian Restaurant
14,Downtown Toronto,1,1,Coffee Shop,Café,Restaurant,Steakhouse,Hotel,American Restaurant,Deli / Bodega,Bar,Gym,Asian Restaurant


<h3> Cluster 3 <h3>

In [163]:
downtoronto_merged.loc[downtoronto_merged['Cluster Label'] == 2, downtoronto_merged.columns[[1] + list(range(5, downtoronto_merged.shape[1]))]]


,Borough,Cluster Label,Cluster Labe,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Downtown Toronto,2,2,Park,Playground,Trail,Building,Yoga Studio,Dessert Shop,Ethiopian Restaurant,Electronics Store,Dumpling Restaurant,Donut Shop


<h3> Cluster 4 <h3>

In [164]:
downtoronto_merged.loc[downtoronto_merged['Cluster Label'] == 3, downtoronto_merged.columns[[1] + list(range(5, downtoronto_merged.shape[1]))]]


,Borough,Cluster Label,Cluster Labe,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Downtown Toronto,3,3,Café,Restaurant,Bar,Japanese Restaurant,Bookstore,Sandwich Place,Bakery,Dessert Shop,Poutine Place,Pub
10,Downtown Toronto,3,3,Vegetarian / Vegan Restaurant,Café,Chinese Restaurant,Bar,Mexican Restaurant,Vietnamese Restaurant,Coffee Shop,Bakery,Dumpling Restaurant,Comfort Food Restaurant


<h3> Cluster 5 <h3>

In [165]:
downtoronto_merged.loc[downtoronto_merged['Cluster Label'] == 4, downtoronto_merged.columns[[1] + list(range(5, downtoronto_merged.shape[1]))]]


,Borough,Cluster Label,Cluster Labe,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,4,4,Grocery Store,Café,Park,Athletics & Sports,Nightclub,Baby Store,Italian Restaurant,Diner,Restaurant,Convenience Store
